# Sea level fingerprints

Today we are going to consider how sea level changes as ice sheets melt. In doing this we will build on the application of the spherical harmonic methods developed last time, and also  consider the iterative solution of an **integral equation** that arises within sea level theory. The full deglacial sea level problem is rather complicated, but we will simplify things substantially by neglecting viscoelastic deformation within the Earth. In spite of this approximation (and a few others  outlined below), the resulting theory is actually very useful in the context of modern day sea level change.

First we import the necessary libraries, read in the data files, and set values for some useful physical parameters. We then plot the 
present day values for sea level and ice thickness

In [ ]:
try:
    import pyslfp as sl
except ImportError: 
    %pip install pyslfp --quiet
    import pyslfp as sl

import numpy as np
import matplotlib.pyplot as plt
import pyshtools as sh
from cartopy import  crs as ccrs


# Set the truncation degree for the calculations
lmax = 256

# Set up the FingerPrint class
fp = sl.FingerPrint(lmax=lmax)
fp.set_state_from_ice_ng()

# Get the present day sea level and ice thickness and plot them
sea_level = fp.sea_level
ice_thickness = fp.ice_thickness

# Set some physical parameters for later use
rho_w = fp.water_density
rho_i = fp.ice_density
g = fp.gravitational_acceleration
b = fp.mean_sea_floor_radius

# Make the plots
fig1, ax1, im1 = sl.plot(
    sea_level,
    colorbar_label="Sea level (m)"
)

fig2, ax2, im2 = sl.plot(
    ice_thickness * fp.ice_projection(),
    cmap = "Blues", 
    colorbar_label="Ice thickness (m)"
)

plt.show()


To understand what is shown above we need to define a number of terms. First, by the **solid Earth** we will mean what remains if we imagine removing all surface water, ice sheets and glaciers, along with the atmosphere. The surface of the 
solid Earth will be called the **solid surface**.  Above each point on the solid surface there may be an ocean of height $S$ and/or an ice sheet of height $I$. Within the context of post-glacial sea level it is sufficient to regard the oceans as being in a state of **hydrostatic equilibrium**, this implying that the **ocean surface** lies on an **equipotential of gravity**. We identify the ocean surface with this equipotential even where  there is no ocean.  Given these definitions, we can now state that the **sea level**, $SL$, is the distance, measured along the local vertical, between the solid surface and the ocean surface. Please note that $SL$ is 
the symbol used for the sea level, and not the product of $S$ with some other variable, $L$. 

Note that because ocean surface is defined globally, it is possible to define sea level both within the oceans, where it is positive, and on land, where it is negative.   A crucial point to remember about sea level is that it is the **relative distance** between the solid and ocean surfaces. It can, therefore, be changed by processes that modify one or both of these surfaces.

In the reality, the ocean surface undergoes short-timescale variations due to winds and tides, and also has long-term topography associated with  ocean currents. Short-timescale variations can have quite large amplitudes (e.g. around 10 metres for tides in some locations), but in discussing sea level change it is  **mean sea level** that is of interest. The topography of the ocean surface caused by ocean currents is typically only a few centimetres in amplitude, and so negligible relative to deglacial changes of order of metres or more. For studies of modern day sea level change, however, variations in ocean currents are significant, as are **steric effects**, this meaning the expansion or contraction of the water column due to variations in  salinity and/or temperature.

## The ocean function


As we have seen, the definition of sea level is such that is is positive where water is present and negative otherwise. It follows that the sea level, $SL$, is not the same as the **ocean thickness**, $S$. To relate the two quantities it is useful to define the **ocean function** as follows
\begin{equation}
C(\mathbf{x}) = \left\{
\begin{array}{cc}
1 & SL(\mathbf{x}) > 0 \\
0 & SL(\mathbf{x}) \le 0
\end{array}
\right.,
\end{equation}
which allows us to write
\begin{equation}
S = C\,SL.
\end{equation}

In fact, this initial definition requires a slight modification to deal with ice shelves. At such a location there is both ice and water present, with the ice floating on top of the water. Using Archimedes principle (i.e. an isostatic balance), for such an ice shelf the following must hold
\begin{equation}
\rho_{w}\, SL  = \rho_{w} \, S + \rho_{i} \,I, 
\end{equation}
where $\rho_{w}$ is the water density and $\rho_{i}$ the ice density. From this relation, we see that the ocean thickness under an ice sheet is given by
\begin{equation}
S = SL - \frac{\rho_{i}}{\rho_{w}}I.
\end{equation}
Because the ocean thickness is non-negative by definition, it follows that
\begin{equation}
\rho_{w}SL - \rho_{i} I \ge 0,
\end{equation}
within an ice shelf. If this condition is violated, then the ice must be **grounded**. This argument neglects flow within the ice, but it is good enough for our purposes. 

Note that the inequality $\rho_{w}SL - \rho_{i} I \ge 0$ actually serves to define where water is present in the more general situation. At locations where $I = 0$, it reduces to our prior condition, while  when $I > 0$ it tells us if there is an ice shelf or a grounded ice sheet. This suggests that we can generalise the definition of the ocean function to
\begin{equation}
C(\mathbf{x}) = \left\{
\begin{array}{cc}
1 & \rho_{w}SL(\mathbf{x})-\rho_{i}I(\mathbf{x}) > 0 \\
0 & \rho_{w}SL(\mathbf{x})-\rho_{i}I(\mathbf{x}) \le 0
\end{array}
\right.,
\end{equation}
and then express the ocean thickness as
\begin{equation}
S = C\left(SL - \frac{\rho_{i}}{\rho_{w}}I\right).
\end{equation}

The ocean function is calculated and stored within the `FingerPrint` object as the property `.ocean_function`.

## A simple approach to melting the Greenland ice sheet

The eventual aim of this practical is to determine the sea level change associated with melting the entire Greenland ice sheet. As a starting point, we can take a very simple approach. We first determine the (signed) mass, $\Delta M_{\mathrm{ice}}$, of the melted ice, calculate the equivalent volume of water, and distribute it uniformly across the oceans. In this manner we find that the predicted sea level change is equal to the constant
\begin{equation}
\Delta SL = -\frac{\Delta M_{\mathrm{ice}}}{\rho_{w}A}, 
\end{equation}
where $A$ is the surface area of the oceans.  Recallng the definition of the ocean function we can write:
\begin{equation}
    A = \int_{\partial M} C \, \mathrm{d} S, \quad \Delta M_{\mathrm{ice}} = \int_{\partial M} \rho_{i} (1-C) \Delta I \, \mathrm{d} S, 
\end{equation}
where the integrals are taken over the Earth's spherical reference surface $\partial M$. Note that in computing the ice mass the factor $(1-C)$ means that we neglect ice shelves. 

To compute the ocean area, $A$, practically we first note that the necessary integral can be written
\begin{equation}
A = b^{2}\int_{\mathbb{S}^{2}} C \,\mathrm{d} S, 
\end{equation}
where $b$ is the Earth's mean radius of $6371$ km. Next, we use the fact that $Y_{00} = 1/\sqrt{4\pi}$ to obtain
\begin{equation}
A = \sqrt{4\pi} b^{2} C_{00}, 
\end{equation}
with $C_{00}$ the $l =0$ and $m =0$ spherical harmonic expansion coefficient of the ocean function. This method 
is implemented within the  `.integrate()` method of the `FingerPrint` class, and we use it to show that the oceans do 
indeed account for about 70% of the Earth's surface. 

In [ ]:
# Compute the ocean area 
ocean_area = fp.integrate(fp.ocean_function)

# Compute the Earth's surface area
surface_area = 4 * np.pi * b * b

# Print the ratio
print(f'Fraction of the surface covered by water {ocean_area/surface_area:.4f}')

With these codes to hand, we can readily estimate the sea level change associated with melting Greenland. The first thing is to set the change, $\Delta I$ in the ice thickness:

In [ ]:
# Use the Greenland projection method to get the desired change
ice_thickness_change = -1 * ice_thickness * fp.greenland_projection(value=0.0)

# Plot the change 
fig, ax, im = sl.plot(
    ice_thickness_change * fp.ice_projection(),    
    colorbar_label="Ice thickness change (m)"
)
plt.show()

We can now compute the necessary integral to find the uniform change in sea level:

In [ ]:
# Compute the uniform change
integrand = rho_i * fp.one_minus_ocean_function * ice_thickness_change
uniform_sea_level_change = -fp.integrate(integrand)/(rho_w * ocean_area)
print(f'Uniform change in sea level {uniform_sea_level_change:.4f} (m)')

## Modelling sea level change

The simple discussion of sea level change above neglects several key physical processes. 

- Ice sheets apply a load to the solid Earth, and also contribute to the Earth's gravitational field. Thus, any changes to ice sheet thickness will cause deformation along with associated gravitational perturbations.

- In the same way, the oceans apply a load to the solid Earth, and contribute to the gravitational field. We cannot, therefore, add new water to the oceans without there being a corresponding response
of the solid Earth.

- The ocean surface lies on an equipotential of gravity, but this potential changes as the Earth deforms. This means that we cannot determine *a prori* the ocean load, but must solve for it as part of the problem.

There are other more sublte effects in modelling sea level change including **shoreline migration** and **rotational feedbacks** that there is not time to consider. 

To understand these ideas quantitatively, let's start by considering the Earth in a state of equilibrium prior to any changes in the ice sheets. At a point $\mathbf{x}$ on the solid surface the sea level, $SL$, is defined implicitly through the requirement
\begin{equation}
\varphi(\mathbf{x} + SL \,\hat{\mathbf{r}}) = \Phi_{G}, 
\end{equation}
where $\varphi$ is the Earth's gravitational potential, $\hat{\mathbf{r}}$ a radial unit vector that defines the local vertical direction, and $\Phi_{G}$ is the *value* of the gravitational potential at the ocean surface. 

Suppose that the Earth is now disturbed by a change  $\Delta I$ to the ice height. Points on the solid surface will  be displaced in response to the combined ice and ocean loads, while the gravitational field will be correspondingly perturbed. Let $\Delta u$ denote the radial component of the surface displacement and $\Delta \varphi$ the gravitational potential perturbation. To first-order accuracy, the implicit definition of sea level
given above can be expanded to yield
\begin{equation}
\Delta SL = - \frac{1}{g}(g \Delta u + \Delta \varphi) +  \frac{\Delta \Phi_{G}}{g}, 
\end{equation}
where $g$ is the acceleration due to gravity prior to the deformation, and $\Delta \Phi_{G}$ denotes the change in the value of the gravitational potential between the new and old equipotential surfaces.  This expression links the deformation of the solid Earth to the associated sea level change. As things stand, however, it is not  useful because it contains the spatially constant term $ \frac{\Delta \Phi_{G}}{g}$ that has not been determined.

The deformation of the solid Earth results from the application of the surface load
\begin{equation}
 \Delta \sigma = \rho_{w} C \Delta SL + \rho_{i} (1-C) \Delta I. 
\end{equation}
Here the first term is that due to changes in ocean height, while the second  is that due to  changes in ice height. If we integrate
the total load over the Earth's surface we obtain
\begin{equation}
    \int_{\partial M} \Delta \sigma \,\mathrm{d} S =  \int_{\partial M} \rho_{w} C \Delta SL \,\mathrm{d} S
    +  \int_{\partial M} \rho_{i} (1-C) \Delta I \,\mathrm{d} S = 0,
\end{equation}
which vanishes because **the mass of water added to the oceans must balance that taken from the ice sheets**. Substituting the above expression for $\Delta SL$ above into this condition we obtain

\begin{equation}
\frac{\Delta \Phi_{G}}{g} = \frac{1}{g A} \int_{\partial M} C \, (g \Delta u + \Delta \varphi) \, \mathrm{d} S
- \frac{\rho_{i}}{\rho_{w} A}\int_{\partial M} (1-C) \Delta I \,\mathrm{d} S. 
\end{equation}
In this manner we have arrived at the following key expression for the change in sea level:

\begin{equation}
\Delta SL = - \frac{1}{g}(g \Delta u + \Delta \varphi) + \frac{1}{g A} \int_{\partial M} C \, (g \Delta u + \Delta \varphi) \, \mathrm{d} S
- \frac{\rho_{i}}{\rho_{w} A}\int_{\partial M} (1-C) \Delta I \,\mathrm{d} S. 
\end{equation}
Here we note that the final term is precisely equal to the constant sea level change we estimated previously.



If we knew both the change in ice height $\Delta I$ along with the response $(\Delta U,\Delta \varphi)$ of the solid Earth we could use this formula to determined the resulting sea level change. The problem is that the response of the solid Earth is determined by the load applied to the solid Earth, and part of this load comes from the sea level change we are trying to determine! 



## Loading Love numbers

To resolve this issue we first  consider how the response of the solid Earth to an applied load can be calculated. A surface load, $\sigma$ acting on the solid Earth has two effects. First it applies a force per unit area (or a **traction**)
\begin{equation}
\mathbf{t} = - \sigma g \hat{\mathbf{r}}, 
\end{equation}
to the solid surface. Note that the sign convention is such that a positive load presses down on the surface. Next, the load has an associated mass, and the corresponding gravitational field applies a force per unit volume (or a **body force**) to the solid Earth. During the resulting deformation the shape and mass distribution in the solid Earth will change, and so its gravitational field will be perturbed. The solid Earth feels its own gravitational field, and hence there is an additional body force in the problem associated with **self-gravitation**.


Even with the simplifying assumption of spherical symmetry, numerical calculation of the resulting deformation is quite complicated. Happily the results can be expressed in a simple and useful manner using **loading Love numbers**. Let us first decompose the applied load using spherical harmonics
\begin{equation}
\Delta \sigma = \sum_{lm} \Delta \sigma_{lm}\, Y_{lm}.
\end{equation}
The radial displacment, $\Delta u$, and gravitational potential perturbation, $\Delta \varphi$, caused by this load can be similarly
expanded as
\begin{equation}
   \Delta u = \sum_{lm} \Delta u_{lm}\, Y_{lm},\quad \Delta \varphi = \sum_{lm} \Delta\varphi_{lm}\, Y_{lm}.
\end{equation}
Because we are dealing with a linearised theory, the deformation due to this load is a sum of the responses associated with the individual terms in the summation.  Moreover, because the solid Earth is assumed to be spherically symmetric it can be shown that the load $\Delta \sigma_{lm} \,Y_{lm}$ produces a response only at the **same degree and order**, such that
\begin{equation}
\Delta u_{lm} = h_{l}\,\Delta \sigma_{lm}, \quad \Delta \varphi_{lm} =  \,k_{l}\,\Delta \sigma_{lm}, 
\end{equation}
where $h_{l}$ and $k_{l}$ are the two **loading Love numbers**. It is worth noting that the Love numbers depend  on spherical harmonic degree but not order.


Calculation of loading Love numbers requires solution of the partial differential equations governing elastic deformation of a self-gravitating planet, and this is quite complicated. The code below simply loads in  pre-computed loading Love numbers calculated for the Earth model PREM of Dziewonski and Anderson (1981). Note that we only show the results for $l \ge 1$. Degree one Love numbers are not needed within this problem as all loads average to zero due to conservation of mass.


In [ ]:
# Get the loading Love numbers
h = fp.h
k = fp.k
deg = np.arange(len(h))

# Make the plots
fig, (ax1, ax2) = plt.subplots(2,1, figsize=(12,8))

ax1.plot(deg[2:], h[2:])
ax1.set_title("Displacement loading Love number (m^3/kg)")

ax2.plot(deg[2:], k[2:])
ax2.set_title("Potential loading Love number (m^4/(kg s^2))")

plt.show()


A key qualitative point is that the amplitude of both the Love numbers decreases rapidly as $l$ increases. This means that the response to a unit load having a long wavelength is larger than that to a unit load with a smaller wavelength. This is the main reason that loading matters for sea level change when an ice sheet melts but is irrelevant if you jump into the ocean. 

## The sea level equation

Let us recall the main equations governing sea level change. First we have 
\begin{equation}
 \Delta \sigma = \rho_{w} C \Delta SL + \rho_{i} (1-C) \Delta I. 
\end{equation}
which expresses the applied surface load in terms of changes to the sea level and ice thickness. As we have noted, it is only the latter that is specified *a priori* in the problem. Next we have 

\begin{equation}
\Delta SL = - \frac{1}{g}(g \Delta u + \Delta \varphi) + \frac{1}{g A} \int_{\partial M} C \, (g \Delta u + \Delta \varphi) \, \mathrm{d} S
- \frac{\rho_{i}}{\rho_{w} A}\int_{\partial M} (1-C) \Delta I \,\mathrm{d} S. 
\end{equation}
which shows how the change in sea level can be computed from the change in ice thickness along with the deformation fields $(\Delta u,\Delta \varphi)$ resulting from the **total** surface load.

If we somehow knew $\Delta SL$, we could calculate $\Delta \sigma$, apply Love numbers to find $(\Delta u,\Delta \varphi)$, and then use the second formula to recompute $\Delta SL$. This is obviously a circular process, but it does suggest an **iterative approach** to the problem:

- Form an initial guess for the sea level change. In practice this might be the uniform sea level change
\begin{equation}
 -\frac{\rho_{i}}{\rho_{w} A}\int_{\partial M} (1-C) \Delta I \,\mathrm{d} S,
\end{equation}
we determined earlier. 

- From the assumed sea level change, calculate the total surface load $\Delta \sigma$, and then use Love numbers to obtain the associated deformation $(\Delta u,\Delta \varphi)$.

- Substitute the computed deformation fields into the expression for sea level to obtained a new estimate
for $\Delta SL$.

- Repeat the whole process, hoping that convergence will be obtained.

Within the remainder of this practical we will implement this idea and see that it really does work. But it is worth thinking through the problem a little more to understand what is happening. First we note that all the equations in the theory of linear. The relationship between the change in sea level $\Delta SL$ and the load $\Delta \sigma$, therefore, takes the form 
\begin{equation}
    \Delta SL = K \,\Delta \sigma, 
\end{equation}
where $K$ is a certain **linear operator** that maps one function linearly into another. The detailed form of $K$ could be determined in terms of spherical harmonics and Love numbers, but this is not necessary. We will, however, note that  concretely this operator can be written

\begin{equation}
(K \,\Delta \sigma) (\theta,\phi) = \int_{\partial M} k(\theta,\phi,\theta',\phi') \, \Delta \sigma(\theta',\phi') \,
\mathrm{d} S', 
\end{equation}
where $k$ is a certain function. Such a linear operator $K$ is said to be an **integral operator** for obvious reasons, while $k$ is known as its **kernel**.

If we substitute the form of the load in the sea level problem into the above equation we find

\begin{equation}
\Delta SL =  K\,[\rho_{i} (1-C)\Delta I] +  K\, [\rho_{w}C\,\Delta SL], 
\end{equation}
where, crucially, we see that $\Delta SL$ occurs on both sides! Thus we arrived at the **sea level equation**, this being an example of a linear **integral equation**. The first term on the right hand side of this equation can be computed directly from the change in ice thickness, and so we set
\begin{equation}
\Delta \tilde{SL} =  K\,[\rho_{i} (1-C)\Delta I].
\end{equation}
To futher simplify notations we also define a new integral operator $\tilde{K}$ by
\begin{equation}
\tilde{K}\,  \Delta SL =  K\,  [\rho_{w}C\,\Delta SL], 
\end{equation}
so that the sea level equation can be written more concisely as
\begin{equation}
\Delta SL = \Delta \tilde{SL}  + \tilde{K} \,\Delta SL.
\end{equation}


One way to solve the sea level equation is to bring $\Delta SL$ entirely over the left hand side as
\begin{equation}
(1-\tilde{K}) \Delta SL = \Delta \tilde{SL}, 
\end{equation}
where $1$ on the left hand side is the identity operator that maps a function to itself. The solution of this equation is then given by

\begin{equation}
\Delta SL = (1-\tilde{K})^{-1} \Delta \tilde{SL}. 
\end{equation}
The necessary inverse operator can then be evaluated  through a so-called **Neumann series**

\begin{equation}
    (1-\tilde{K})^{-1} = \sum_{n=0}^{\infty} \tilde{K}^{n} = 1 + \tilde{K} + \tilde{K}^{2} + \tilde{K}^{3} + \cdots, 
\end{equation}
which can be seen as a generalisation of a binomial expansion to linear operators. Convergence of such an expansion is not assured, but it can be shown to hold so long as $\tilde{K}$ is small in an appropriate sense. 

The iterative scheme for solving the sea level equation takes the form

\begin{equation}
\Delta SL_{i+1} = \Delta \tilde{SL} + \tilde{K} \Delta SL_{i}, 
\end{equation}
for $i = 0,1,2,\dots$, with $\Delta SL_{0}$ being some initial guess. For the first iteration we have

\begin{equation}
\Delta SL_{1} = \Delta \tilde{SL} + \tilde{K} \Delta SL_{0}.
\end{equation}
Next we find

\begin{align}
\Delta SL_{2} &= \Delta \tilde{SL} + \tilde{K} \Delta SL_{1}, \\
              &=  \Delta \tilde{SL}  + \tilde{K} [ \Delta \tilde{SL} + \tilde{K} \Delta SL_{0}], \\
              &=  \Delta \tilde{SL}  + \tilde{K} \Delta \tilde{SL} + \tilde{K}^{2} \Delta SL_{0}, 
\end{align}
while the third is similarly found to be

\begin{align}
\Delta SL_{3} &=  \Delta \tilde{SL}  + \tilde{K} \Delta \tilde{SL} + \tilde{K}^{2} \Delta \tilde{SL}
+ \tilde{K}^{3} \Delta SL_{0}.
\end{align}

Noting the pattern, the $p$ th term is given by

\begin{align}
\Delta SL_{p} &=  \sum_{n=0}^{p-1} \tilde{K}^{n}\Delta \tilde{SL}  
+ \tilde{K}^{p} \Delta SL_{0}, 
\end{align}
and so letting $p$ increase the result tends to the Neumann series so long as the condition 
\begin{equation}
\lim_{n\rightarrow \infty}\tilde{K}^{n} (\Delta SL_{0}) = 0,
\end{equation}
holds for any $\Delta SL_{0}$. It can be shown that this is prescisely the requirement for the 
Neumann series to converge, and hence the iterative method for solving the sea level equation has a solid theoretical foundation.

Within the code below, we define two useful functions. The first takes in a load and returns the resulting vertical displacement 
and gravitational potential change. The second then computes the sea level change given the solid Earth deformation along 
with the associated direct load, $\zeta = \rho_{i}(1-C)\Delta I$. Using these two functions, you will build your own iterative solver 
for the sea level equation within the practical. 

In [ ]:
def load_to_deformation(load):
    """
    Given a load, returns the vertical displacement and gravitational 
    potential perturbation. 
    
    This routine assumes that various physical parameters have been defined
    in the code cells above.
    """

    # Transform the load
    load_coefficients = load.expand(normalization="ortho")

    # Set up the displacement and potential coefficients
    displacement_coefficients = load_coefficients.copy()
    potential_coefficients = load_coefficients.copy()

    # Scale by the Love numbers
    for l in range (load.lmax+1):
        displacement_coefficients.coeffs[:, l, :] *= h[l]
        potential_coefficients.coeffs[:, l, :] *= k[l]
        
    # Expand the coefficients -- we use a FingerPrint method that 
    # ensures the grid parameters are consistent with those for the 
    # class instance. 
    displacement = fp.expand_coefficient(displacement_coefficients)
    potenital_change = fp.expand_coefficient(potential_coefficients)

    return displacement, potenital_change

def deformation_to_sea_level(displacement, potenital_change, direct_load):
    """
    Given the vertical displacement, potential change, and direct load, 
    this function returns the associated sea level change. 
    """

    # Compute the uniform sea level change
    uniform_sea_level_change = - fp.integrate(direct_load) / (rho_w * ocean_area)

    # Compute the spatially variable sea level change
    sea_level_change = -1 * (displacement + potenital_change/g)

    # Correct the mean of the spatially variable part
    sea_level_change.data[:,:] -= fp.integrate(sea_level_change) / ocean_area
    sea_level_change.data[:,:] += uniform_sea_level_change

    return sea_level_change


# Set the direct load
direct_load = rho_i * fp.one_minus_ocean_function * ice_thickness_change

# Compute the deformation
displacement, potenital_change = load_to_deformation(direct_load)

# Compute the sea level change
sea_level_change = deformation_to_sea_level(displacement, potenital_change, direct_load)

# Plot the sea level change within the oceans
fig, ax, im = sl.plot(
    fp(direct_load=direct_load, rotational_feedbacks=False)[0] * fp.ocean_projection(), 
    colorbar_label = "Sea level change (m)"
)
plt.show()

## What you need to know and be able to do

- Explain what is meant by sea level along with associated definitions such as the ocean and solid surfaces.

- Derive the formula used to estimate globally uniform sea level change.

- Explain how loading Love numbers can be used to compute the deformation associated with a given load, and outline how this can be done numerically. Here the emphasis should be on the key steps involved, and not on the details of any of the algorithms.

- Explain qualitatively how the sea level equation is obtained. Here you do not need to reproduce any of the detailed formulae, but you might be asked to carrry out steps from the derivation given the necessary information (e.g. show how conservation of mass is used to determine the uniform term within the sea level change).
- What an integral equation is, and how it can be solved in an iterative manner. You do not need to prove why the Neumann series converges, but should be familiar with its form.
- describe why iterative methods are useful in solving the sea level equation and explain in outline the steps necessary to do this, 


## Practical Exercise 1:

Using the methods described in the lecture, implement an iterative solution of the sea level equation. You should write a function 
to do this that takes in a direct load
$$
\zeta = \rho_{i}(1-C) \Delta I, 
$$
corresponding to the chosen ice thickness change, and then returns the corresponding change of sea level. 

You should implement a suitable criterion by which the iterative has coverged to a suitable accuracy. 

To check your method, you can compute the sea level change directly using the `FingerPrint` instance as follows:

```python
sea_level_change, _, _, _ = fp(direct_load=direct_load, rotational_feedbacks=False)
```

The other arguments returned by this function are the vertical displacement, the gravity potential change, and the perturbation 
to the Earth's rotation vector. We have not considered rotational feedbacks in this lecture, and so this feature should be 
turned off to make a comparison with your own code. 

Once you have a working code, you can look at the sea level changes associated with different ice sheets. You can conveniently 
obtain appropriate directly loads using the methods `fp.greenland_load`, `fp.west_antarctic_load`, and  `fp.east_antarctic_load`.

In [ ]:
def solve_sea_level_equation(direct_load, tol=1e-5, max_iter=10):
    """
    Solves the sea level equation iteratively for a given direct ice load.
    
    Parameters:
    direct_load: The initial ice load ζ = ρi * (1-C) * ΔI.
    tol: The convergence tolerance (m).
    max_iter: Maximum number of iterations to prevent infinite loops.
    """
    
    # Initial guess: The uniform sea level change
    # We use the deformation_to_sea_level function with zero deformation
    # to get the initial uniform term.
    zero_grid = direct_load.copy()
    zero_grid.data[:] = 0.0
    sl_estimate = deformation_to_sea_level(zero_grid, zero_grid, direct_load)
    
    for i in range(max_iter):
        sl_old = sl_estimate.copy()
        
        # Calculate the total surface load: Δσ = ρw * C * ΔSL + ζ
        # Note: direct_load already contains the (1-C) factor.
        ocean_load = rho_w * fp.ocean_function * sl_old
        total_load = ocean_load + direct_load
        
        # Calculate deformation from total load
        displacement, potential_change = load_to_deformation(total_load)
        
        # Compute the new sea level estimate
        sl_estimate = deformation_to_sea_level(displacement, potential_change, direct_load)
        
        # Check for convergence
        # We look at the maximum absolute difference between iterations
        diff = np.max(np.abs(sl_estimate.data - sl_old.data))
        print(f"Iteration {i+1}: Max change = {diff:.2e} m")
        
        if diff < tol:
            print("Converged!")
            break
    else:
        print("Warning: Maximum iterations reached without full convergence.")
        
    return sl_estimate

# --- Run the Solver ---
# Define the direct load from the Greenland ice thickness change
direct_load = rho_i * fp.one_minus_ocean_function * ice_thickness_change

# Compute the solution using our iterative solver
my_sl_change = solve_sea_level_equation(direct_load)

# --- Verification ---
# Compare with the built-in FingerPrint solver
expected_sl_change, _, _, _ = fp(direct_load=direct_load, rotational_feedbacks=False)

# Compute the difference between our answer and the benchmark
error = np.max(np.abs(my_sl_change.data - expected_sl_change.data)) 
error /= np.max(np.abs(expected_sl_change.data))
print(f"\nMax relative difference from benchmark: {error:.2e}")

# Plot the final sea level fingerprint
fig, ax, im = sl.plot(
    my_sl_change * fp.ocean_projection(), 
    colorbar_label="Iterative Sea Level Change"
)
plt.show()